## 多通道输入
**每一个通道都有一个卷积核，结果是所有通道卷积结果的和->结果是单通道**
![softmax-description](./imgs/21-1.png)
公式如下：
这里输入和卷积核的channel是相同的
![softmax-description](./imgs/21-2.png)

## 多通道输出
目前，无论有多少输入通道，到目前为止我们都只能得到单输出通道

我们可以有多个三维卷积核，每个卷积核生成一个输出通道。
![softmax-description](./imgs/21-3.png)

为什么要采用多通道输出？
- 每一个输出通道可以识别特定模式

## 1 * 1卷积层
- 它不识别空间模式，只是融合通道。
![softmax-description](./imgs/21-4.png)
- 1 * 1的卷积等价为一个全连接层
- 之前没有认真理解过全连接层，这里记一些笔记：
- 多层感知机的每一个hidden layer都是一个全连接层
- 这里其实还是不太懂，等我复习的时候再去理解。

## 二维卷积层
关于bias: 每一个输入通道和每一个输出通道都有一个偏差。

10层，1M样本，10PFlop（而且只是前向）
![softmax-description](./imgs/21-5.png)

In [21]:
import torch
import d2l
import torch.nn as nn

def corr2d_multi_in(X, K):
    # 先遍历 “X” 和 “K” 的第0个维度（通道维度），再把它们加在一起
    # zip: 拆分第一个>1的维度，拆成维度为1并打包（zip有进行降维）
    # 比如这里： X: 2, 3, 3, K: 2, 2, 2
    # 那么x:3, 3, k: 2, 2
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [22]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
# 进行多输入通道卷积
'''
# 这一句可以查看zip的尺寸
for x, k in zip(X, K):
    print(x.size(),k.size())
    break
'''
X.size(),K.size(),corr2d_multi_in(X, K)

(torch.Size([2, 3, 3]),
 torch.Size([2, 2, 2]),
 tensor([[ 56.,  72.],
         [104., 120.]]))

In [23]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    # 注意：第0维默认是output channel的维度
    # c_o *  c_i * h * w
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [24]:
K = torch.stack((K, K + 1, K), 0) #拓展第0维度

In [25]:
for k in K: # for k in K: 拆第0维
    print(k)
    print(k.size())
    break
K.shape

tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])
torch.Size([2, 2, 2])


torch.Size([3, 2, 2, 2])

In [26]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 56.,  72.],
         [104., 120.]]])

In [29]:
# 这里为了证明： 1 * 1的卷积 等价于一个全连接层
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape # 
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [32]:
X = torch.normal(0, 1, (3, 3, 3)) # in-channel = 3
K = torch.normal(0, 1, (2, 3, 1, 1)) # out-channel = 2, in-channel = 3

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6 #表达式为false时触发异常

In [36]:
# output channel, input channel
conv2d = nn.Conv2d(1,1,kernel_size = 3, padding =1, stride = 2)
conv2d = nn.Conv2d(1,1,kernel_size = (3,5), padding =(0,1), stride = (2,4))